# Demonstration of shifty.py's "downloader" module
 - 20191106
 - Payne 
 
As the name suggests, the downloader module will download the raw data used in shifty. 

There is a "parent" class" called "Downloader".
 - This is NOT likely to be used directly at all 

There are "child" classes for each type of data that will be downloaded:
 - These ARE likely to be directly used in practice. 
 
It intended that there be a specific downloader for a specific kind of data.
E.g.
 - The "TESSDownloader" class provides methods to download TESS data (from mast)
 - The "HSTDownloader" class is just a stub and does not yet have much functionality

This notebook:
 - Demonstrates the key components / classes / functions available in the downloader module
 - Acts as a complement to the tests of the same module. 

#### Import statements 

In [1]:
# -------------------------------------------------------------------------------------
# Third party imports
# -------------------------------------------------------------------------------------
import numpy as np 
import matplotlib.pyplot as plt
import os, sys
import glob 

import astropy
from astropy.io import fits
from astropy.time import Time
from astropy.units import allclose as quantity_allclose
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates.builtin_frames import FK5, ICRS, GCRS, GeocentricMeanEcliptic, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, GeocentricTrueEcliptic, BarycentricTrueEcliptic, HeliocentricTrueEcliptic, HeliocentricEclipticIAU76
from astropy.constants import R_sun, R_earth

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
sys.path.append( os.path.join(os.path.dirname(os.getcwd() ), 'shifty')  )
import downloader 

### Instantiate TESSDownloader object
 - Note that the first time this is done will cause the creatation of storage directories on your local disk. 

In [2]:
# creation of TESSDownloader object
T = downloader.TESSDownloader()

# examination of instantiated variables
# - these are defining some local data storage 
for k,v in T.__dict__.items():
    print(k,v)
    
# check that storage directories exist on disk 
for d in [T.local_dir , T.tess_dir]:
    print( 'It is *%r* that %r exists on disk' % (os.path.isdir(d), d)  )

local_dir /Users/matthewjohnpayne/.shifty_data
tess_dir /Users/matthewjohnpayne/.shifty_data/tess
It is *True* that '/Users/matthewjohnpayne/.shifty_data' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess' exists on disk


### Use TESSDownloader object to download a small set of ~10 TESS fits files
 - This is intended to be used when doing quick development work when you only want to work on a small set of data
 
Files will be put into a directory structure that looks like ...
 - self.tess_dir/sectorNumber/cameraNumber/chipNumber/tess2018292095940-s0004-1-1-0124-s_ffic.fits
 - E.g. self.tess_dir/4/1/1/tess2018292095940-s0004-1-1-0124-s_ffic.fits
 - N.B. If individual files have already been downloaded, they will *not* be re-lownloaded


In [3]:
# download the test data 
T._ensure_test_data_available_locally()

# check that the expected data actually exists 
expected_filepaths = T._define_test_data()
for fp in expected_filepaths:
    print( 'It is *%r* that %r exists on disk' % (os.path.isfile(fp) , fp)  )

It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292095940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292102940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292105940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292112940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292115940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292122940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292125940-s0004-1-1-0124-s_ffic.fits' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/1/1/tess2018292132940-s0004-1-1-0124-s_ff

### Use TESSDownloader object to download all data for a single chip on a single camera for a single sector of TESS data
 - This will download ~1,300 files 
 
 - #### This takes a long time!
 

Files will be put into a directory structure that looks like ...
 - self.tess_dir/sectorNumber/cameraNumber/chipNumber/tess2018292175940-s0004-1-2-0124-s_ffic.fits
 - E.g. self.tess_dir/4/1/2/tess2018292175940-s0004-1-2-0124-s_ffic.fits
 - N.B. If individual files have already been downloaded, they will *not* be re-lownloaded


In [4]:
# download the data 
sectorNumber, cameraNumber, chipNumber = 4,3,2
T.download_chip(sectorNumber, cameraNumber, chipNumber)

# check that the expected destination directory is now populated with lots of files ...
expectedDirectory = os.path.join(T.tess_dir, str(sectorNumber), str(cameraNumber), str(chipNumber) )
print( 'It is *%r* that %r exists on disk' % (os.path.isdir(expectedDirectory) , expectedDirectory)  )

fits_files_in_expectedDirectory = glob.glob( expectedDirectory + '/*.fits')
print( 'There are %d fits files in expectedDirectory' % len(fits_files_in_expectedDirectory))

It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess/4/3/2' exists on disk
There are 1060 fits files in expectedDirectory


### Use TESSDownloader object to download all data for an entire single sector of TESS data

In [14]:
# download the data 
sectorNumber, cameraNumbers, chipNumbers = 5,[1,2,3,4],[1,2,3,4]

for cameraNumber in cameraNumbers:
    for chipNumber in chipNumbers:
        T.download_chip(sectorNumber, cameraNumber, chipNumber)


In [3]:
# download the data 
sectorNumber, cameraNumbers, chipNumbers = 18,[1,2,3,4],[1,2,3,4]

for cameraNumber in cameraNumbers:
    for chipNumber in chipNumbers:
        T.download_chip(sectorNumber, cameraNumber, chipNumber)


### Use TESSDownloader object to download pixel response function models data for TESS
 - from  https://archive.stsci.edu/missions/tess/models/prf_fitsfiles/

for documentation & description:
 - see https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview#id-2.0-DataProductOverview-PixelResponseFunctions


In [5]:
# getting prf data
# -----------------------------------------
T.download_prf()

# we expect the files to have been extracted into ...
# "~/.shifty_data/prf/start_000<X>/cam<Y>_ccd<Z>" where X=1or4, Y=1-to-4, & Z=1-to-4
assert os.path.isdir( T._fetch_tess_prf_directory() )
for sectorString in ['start_s0001', 'start_s0004']:
    for i in range(1,5):
        for j in range(1,5  ):
            filepath = os.path.join( T._fetch_tess_prf_directory() ,
                                    sectorString,
                                    'cam%d_ccd%d' % (i,j)  )
            # check directory exists & contains >= 25 files
            print( 'it is %r that the %r directory exists and contains more than 25 fits-files' % 
                  (os.path.isdir( filepath ) and len(glob.glob(filepath + '/*fits') ) >= 25, filepath))



it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam1_ccd1' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam1_ccd2' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam1_ccd3' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam1_ccd4' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam2_ccd1' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam2_ccd2' directory exists and contains more than 25 fits-files
it is True that the '/Users/matthewjohnpayne/.shifty_data/tess/prf/start_s0001/cam2_ccd3' directory exists and contains more than 25 fit